<a href="https://colab.research.google.com/github/yolile/paraguay_oncology_medicines_ocds_data/blob/main/2022_Datos_Licitaciones_Oncol%C3%B3gicas_MSBSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descarga
Descargamos todos los procesos de contratación desde Enero del 2015 a Julio 2022, que dentro de los productos licitados tenga uno de los listados en la Lista de Medicamentos Escenciales oncológicos del Ministerio de Salud

In [10]:
# Listado de medicamentos oncologicos del LME con su código clasificador nivel 4
# de las naciones unidas utilizado para las compras realizadas a través de la DNCP
lme = [
  {'nombre': 'Ciclofosfamida', 'codigo': '51111507'},
  {'nombre':'Ifosfamida', 'codigo': '51111509'},
  {'nombre':'Metotrexato', 'codigo': '51111610'},
  {'nombre':'Mercaptopurina', 'codigo': '51111609'},
  {'nombre':'Citarabina', 'codigo': '51111602'},
  {'nombre':'Fluorouracilo', 'codigo': '51111605'},
  {'nombre':'Sulfato de vinblastina', 'codigo': '51111812'},
  {'nombre':'Sulfato de vincristina', 'codigo': '51111813'},
  {'nombre':'Etoposide', 'codigo': '51111614'},
  {'nombre':'Paclitaxel', 'codigo': '51111904'},
  {'nombre':'Clorhidrato de doxorubicina', 'codigo': '51111711'},
  {'nombre':'Sulfato de bleomicina', 'codigo': '51111701'},
  {'nombre':'Cisplatino', 'codigo': '51111506'},
  {'nombre':'Carboplatina', 'codigo': '51111503'},
  {'nombre':'Agentes Alquilantes varios', 'codigo': '51111501'},
  {'nombre':'Asparaginasa', 'codigo': '51111699'},
  {'nombre':'Hidroxiurea', 'codigo': '51111606'},
  {'nombre':'Tamoxifeno', 'codigo': '51111809'},
  {'nombre':'Anastrozol', 'codigo': '51111801'},
  {'nombre':'Filgrastim', 'codigo': '51201802'},
  {'nombre':'Mofetil micofenolato', 'codigo': '51201503'},
  {'nombre':'Ciclosporina', 'codigo': '51201502'}
]

Utilizamos el buscador de licitaciones de la DNCP que permite filtrar por fecha y productos

In [30]:

url_buscador_licitaciones = 'https://contrataciones.gov.py/buscador/licitaciones.csv?nro_nombre_licitacion=&fecha_desde=01-01-2015&fecha_hasta=31-07-2022&tipo_fecha=PUB'
url_buscador_licitaciones += '&convocante_tipo=&convocante_nombre_codigo=&codigo_contratacion=&catalogo%5Bcodigos_catalogo_n4%5D={0}&catalogo%5Bcodigos_catalogo_n4_label%5D={1}'
url_buscador_licitaciones += '&page=&order=&convocante_codigos=&convocante_tipo_codigo=&unidad_contratacion_codigo='

In [33]:
import pandas as pd
import urllib
licitaciones = pd.DataFrame()
for medicamento in lme:
  url = url_buscador_licitaciones.format(medicamento['codigo'], urllib.parse.quote(medicamento['nombre']))
  licitaciones_tmp = pd.read_csv(url, sep=';')
  licitaciones = pd.concat([licitaciones, licitaciones_tmp])

Borramos cualquier licitacion duplicada que se pudo encontrar (por ejemplo donde la misma licitacion compre más de un producto oncologico)

In [ ]:
licitaciones.drop_duplicates(inplace=True)

In [36]:
licitaciones['fecha_publicacion_convocatoria'] =  pd.to_datetime(licitaciones['fecha_publicacion_convocatoria'])
licitaciones['anio'] = licitaciones['fecha_publicacion_convocatoria'].dt.year

Descargamos los detalles completos de todos los procesos de licitación en formato OCDS

In [48]:
!mkdir data

In [ ]:
import requests
import json
import time

def get_ocds(ocid):
  status = 429
  if ocid is not None:
    while status == 429: 
      ocid = int(ocid)
      response = requests.get(f'http://www.contrataciones.gov.py/datos/api/v3/doc/ocds/record/ocds-03ad3f-{ocid}')
      data = response.json()
      status = response.status_code
      print(ocid, status)
      if status == 429:
        time.sleep(5)
        continue
      if status == 404:
        continue
      data = data['records'][0]['compiledRelease']
      with open(f'data/{ocid}.json', 'w') as f:
        json.dump(data, f)

licitaciones['nro_licitacion'].apply(get_ocds)

In [ ]:
# Instalamos unas herramientas para convertir los archivos JSON a CSV
!pip install flattentool
!pip install ocdskit
# Unimos todos los JSON en uno solo
!cat data/*.json | ocdskit package-releases > data.json
# Lo convertimos en archivos CSVs
!flatten-tool flatten data.json --root-id=ocid -f csv -s https://standard.open-contracting.org/schema/1__1__5/release-schema.json --main-sheet-name releases --root-list-path=releases   --remove-empty-schema-columns

# Estadísticas preliminares

## Cuántos procesos
**Se encontraron un total de 127 procesos licitatorios que compran medicamentos oncologicos de la lista de LME desde el 2015**

In [50]:
len(licitaciones)

127

## Quiénes compran?
Por número de procesos, mayormente el Ministerio de Salud, IPS, la UNA, la Policía Nacional y la DIBEN.

In [45]:
licitaciones.groupby(['convocante']).count()['nro_licitacion'].reset_index().sort_values(['nro_licitacion'], ascending=False)

,convocante,nro_licitacion
11,Ministerio de Salud Pública y Bienestar Social...,39
9,Instituto de Previsión Social / Entidades Públ...,27
4,Facultad de Ciencias Medicas / Universidad Nac...,20
12,Policia Nacional / Ministerio del Interior / P...,10
13,Secretaria Nacional por los Derechos Humanos d...,9
1,Dirección de Beneficencia / Entes Autónomos y...,7
3,Dirección de Beneficencia / Entes Autónomos y ...,3
0,Comando en Jefe Uoc 1 / Ministerio de Defensa ...,2
5,Facultad de Ciencias Quimicas / Universidad Na...,2
7,Gobierno Departamental de Concepción / Gobiern...,2


## Qué tipo de procesos se utilizan?
**Mayormente Licitaciones Públicas Nacionales y Contrataciones por Excepción**

In [47]:
licitaciones.groupby(['tipo_procedimiento']).count()['nro_licitacion'].reset_index().sort_values(['nro_licitacion'], ascending=False)

,tipo_procedimiento,nro_licitacion
3,LPN - Licitación Pública Nacional,81
1,CE - Contratación por Excepción,31
2,CO - Concurso de Ofertas,8
0,CD - Contratación Directa,7


## Análisis competencia

Preparamos los datos:
Juntamos la lista de items adjudicados con la de proveedores y clasificaciones

In [100]:
proveedores_adjudicados = pd.read_csv('flattened/awa_suppliers.csv')
adjudicaciones = pd.read_csv('flattened/awards.csv')
items = pd.read_csv('flattened/awa_items.csv')
classificacion_n4 = pd.read_csv('flattened/awa_ite_additionalClassificatio.csv')
tenders = pd.read_csv('flattened/releases.csv')

In [106]:
adjudicados = pd.merge(tenders, adjudicaciones, on='ocid')
adjudicados = pd.merge(adjudicados, proveedores_adjudicados, on='awards/0/id')
adjudicados = pd.merge(adjudicados, items, on='awards/0/id')
adjudicados = pd.merge(adjudicados, classificacion_n4, on='awards/0/items/0/id')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ocid_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.


Filtramos solo los items oncológicos

In [107]:
lme_dataframe = pd.DataFrame(lme)
lme_dataframe['codigo'] = lme_dataframe['codigo'].astype('int')
items_adjudicados_oncologicos = adjudicados[adjudicados['awards/0/items/0/additionalClassifications/0/id'].isin(lme_dataframe['codigo'].tolist())]
items_adjudicados_oncologicos = items_adjudicados_oncologicos.drop_duplicates()
items_adjudicados_oncologicos['precio_x_cantidad'] = items_adjudicados_oncologicos['awards/0/items/0/unit/value/amount']*items_adjudicados_oncologicos['awards/0/items/0/quantity']

## Cuántas adjudicaciones?
337 adjudicaciones, para 38 proveedores diferentes

In [108]:
len(items_adjudicados_oncologicos['awards/0/id_x'].unique())

337

In [109]:
len(items_adjudicados_oncologicos['awards/0/suppliers/0/id'].unique())

38

In [110]:
items_adjudicados_oncologicos

,ocid_x,id_x,tender/id,tender/coveredBy,tender/title,tender/status,tender/awardCriteria,tender/awardCriteriaDetails,tender/submissionMethod,tender/techniques/hasElectronicAuction,...,awards/0/items/0/unit/value/currency,awards/0/items/0/relatedLot,ocid_y,id,awards/0/id_y,awards/0/items/0/additionalClassifications/0/id,awards/0/items/0/additionalClassifications/0/scheme,awards/0/items/0/additionalClassifications/0/description,awards/0/items/0/additionalClassifications/0/uri,precio_x_cantidad
668,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-lpn-17-15-adquisicion-medicamentos-vari...,NaN,LPN 17-15 ADQUISICION DE MEDICAMENTOS VARIOS P...,complete,priceOnly,Por Item,electronicAuction,True,...,PYG,gjhmc1pQjEs=,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-prosaludfarma-s-a-67,51111609,UNSPSC,Mercaptopurina,https://www.contrataciones.gov.py/datos/api/v3...,252000000
669,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-lpn-17-15-adquisicion-medicamentos-vari...,NaN,LPN 17-15 ADQUISICION DE MEDICAMENTOS VARIOS P...,complete,priceOnly,Por Item,electronicAuction,True,...,PYG,n2Z1TK3ejKs=,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-prosaludfarma-s-a-67,51111606,UNSPSC,Hidroxiurea,https://www.contrataciones.gov.py/datos/api/v3...,226800000
724,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-lpn-17-15-adquisicion-medicamentos-vari...,NaN,LPN 17-15 ADQUISICION DE MEDICAMENTOS VARIOS P...,complete,priceOnly,Por Item,electronicAuction,True,...,PYG,Sd16trY+Kfs=,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-bioethic-pharma-s-a-102,51201502,UNSPSC,Ciclosporina,https://www.contrataciones.gov.py/datos/api/v3...,1251150000
725,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-lpn-17-15-adquisicion-medicamentos-vari...,NaN,LPN 17-15 ADQUISICION DE MEDICAMENTOS VARIOS P...,complete,priceOnly,Por Item,electronicAuction,True,...,PYG,gAzMBoaE99E=,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-bioethic-pharma-s-a-102,51111701,UNSPSC,Sulfato de bleomicina,https://www.contrataciones.gov.py/datos/api/v3...,105000000
732,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-lpn-17-15-adquisicion-medicamentos-vari...,NaN,LPN 17-15 ADQUISICION DE MEDICAMENTOS VARIOS P...,complete,priceOnly,Por Item,electronicAuction,True,...,PYG,MvcQGg2YPGU=,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-fusa-s-a-76,51111812,UNSPSC,Sulfato de vinblastina,https://www.contrataciones.gov.py/datos/api/v3...,87000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9866,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-adquisicion-medicamentos-oncologicos-ca...,NaN,ADQUISICIÓN DE MEDICAMENTOS ONCOLÓGICOS CABAZI...,complete,priceOnly,Por Total,electronicAuction,True,...,PYG,K27yxpp97QU=,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-mather-company-srl-3,51111904,UNSPSC,Paclitaxel,https://www.contrataciones.gov.py/datos/api/v3...,2018362424
9867,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-adquisicion-medicamentos-oncologicos-ca...,NaN,ADQUISICIÓN DE MEDICAMENTOS ONCOLÓGICOS CABAZI...,complete,priceOnly,Por Total,electronicAuction,True,...,PYG,K27yxpp97QU=,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-bioethic-pharma-s-a-5,51111904,UNSPSC,Paclitaxel,https://www.contrataciones.gov.py/datos/api/v3...,2018362424
9868,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-adquisicion-medicamentos-oncologicos-ca...,NaN,ADQUISICIÓN DE MEDICAMENTOS ONCOLÓGICOS CABAZI...,complete,priceOnly,Por Total,electronicAuction,True,...,PYG,K27yxpp97QU=,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...

Descargamos el CSV final con toda la información necesaria

In [111]:
items_adjudicados_oncologicos.to_csv('final.csv')

In [96]:
adj_monto = items_adjudicados_oncologicos.groupby('awards/0/suppliers/0/name').sum('precio_x_cantidad').sort_values('precio_x_cantidad', ascending=False).reset_index()

adj_monto['porcentaje'] = adj_monto['precio_x_cantidad']/items_adjudicados_oncologicos['precio_x_cantidad'].sum()*100

In [87]:
adj_monto

,awards/0/suppliers/0/name,awards/0/value/amount,awards/0/items/0/quantity,awards/0/items/0/unit/value/amount,awards/0/items/0/additionalClassifications/0/id,precio_x_cantidad,porcentaje
0,EUROQUIMICA S.A.,293413261490,4177834,12738196,2046802341,105370453160,16.363069
1,Bioethic Pharma S.A.,207804115120,3361250,15191113340,1842087144,95273486310,14.795102
2,QUIMFA S.A.,127501762769,3952680,7878778,1074963084,77786402700,12.079518
3,FARMACEUTICA PARAGUAYA S.A.,679430331712,8516039,95537455,8484631766,50945446569,7.911362
4,PROSALUD FARMA S.A.,248095680467,2010886,18862327,2402787948,37028139042,5.750132
5,FUSA S.A.,355854689267,377336,37321779,8484531762,33092279588,5.138929
6,EXXEL TECHNOLOGIES S.A,58633021600,1763680,177559,563216533,27510917600,4.272194
7,INSUMOS MEDICOS S.A.,220892110434,929316,25774618,1431755817,26083134340,4.050472
8,MATHER COMPANY SRL,178996079194,88599,6085624369,2760028028,23804294094,3.696589
9,EUROTEC S.A.,79911604860,77855,13957244,715742459,21754270350,3.378239
